In [3]:
search_for = 128
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.011354923248291016
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 330
all cases: 866251507
type: [1, 1, 1, 1, 128] 128
cases of this type: 268435456
100000 31.67310872642061
200000 55.34604309208903
300000 65.67476503344722
400000 71.47747475043222
500000 67.63249726133122
600000 71.8677086988205
700000 67.1245379600409
800000 66.9415345072082
900000 66.76847929996248
1000000 67.14808657307549
1100000 20.91160176242439
1200000 68.22806848070805
1300000 65.72962242442622
1400000 65.24519168637978
1500000 65.71601952041013
1600000 68.62864758915549
1700000 66.4753822974783
1800000 65.3330157559475
1900000 67.00096792506206
2000000 65.96381654265294
2100000 52.219944357352304
2200000 36.16634715922544
2300000 64.39212259980602
2400000 65.99668858432128
2500000 63.70481345450866
2600000 64.09133985434441
2700000 66.47168843947397
2800000 63.67937011238449
2900000 64.52726834972697
3000000 63.2876118765408
31000

30200000 70.13421063560442
30300000 66.24108384805182
30400000 55.627412281388274
30500000 38.460294574022356
30600000 69.12886335734879
30700000 72.89055898026199
30800000 67.63570585132345
30900000 68.75080662107653
31000000 76.7812867631023
31100000 73.52568850855917
31200000 73.57954767033027
31300000 37.20889627302732
31400000 73.77879508245368
31500000 74.89580875022098
31600000 76.74635862815472
31700000 57.611025623257746
31800000 51.72555897034664
31900000 77.29450257369828
32000000 75.50226901755018
32100000 73.04153441163872
32200000 74.35885937562558
32300000 69.55408452152116
32400000 67.66945641546765
32500000 56.7034524622015
32600000 42.88156094761817
32700000 68.3312695155011
32800000 74.27808692934389
32900000 71.60739906424783
33000000 71.7132288706564
33100000 71.333562955463
33200000 76.08389652143373
33300000 62.141344310492755
33400000 44.74920777005971
33500000 74.34089374344886
33600000 76.16802079372367
33700000 75.56009969734463
33800000 68.09679078467303
339

60700000 74.74117610435877
60800000 78.489561624761
60900000 81.72341730093838
61000000 79.39391257015076
61100000 77.04470353120713
61200000 74.10023861618275
61300000 53.3757659510356
61400000 49.40377230897676
61500000 70.62685504761247
61600000 66.08387242059555
61700000 66.7970409943165
61800000 67.24154009089342
61900000 31.11350669669976
62000000 64.46108233249544
62100000 66.620263730138
62200000 67.96948616515681
62300000 68.78418583675085
62400000 68.02215642437051
62500000 33.930116699307675
62600000 73.6396120436974
62700000 77.64456682639685
62800000 77.5857860799636
62900000 80.21797556518312
63000000 74.76720944099792
63100000 72.87725116877274
63200000 70.94550812190198
63300000 76.75715557708624
63400000 60.44577684067397
63500000 43.05422925322567
63600000 65.69775109497976
63700000 67.66771814364655
63800000 67.60475417888209
63900000 66.80474912792074
64000000 29.140992528903443
64100000 66.30893449977779
64200000 69.23129088370813
64300000 65.00912906859965
6440000

91200000 62.779050600556324
91300000 25.484545989264337
91400000 64.1774161060804
91500000 66.34449980885472
91600000 38.26749030641715
91700000 59.569276278005894
91800000 61.4536682172277
91900000 65.71245840586701
92000000 65.64371093210679
92100000 71.24849575992461
92200000 69.85414588053983
92300000 72.14241862991119
92400000 69.47058333467413
92500000 72.98889964324651
92600000 69.74903694849648
92700000 67.55454225253705
92800000 66.68411491852378
92900000 62.458206903379875
93000000 46.53230291251463
93100000 43.060984470343385
93200000 64.87160825539291
93300000 60.117867995922175
93400000 27.253497190354512
93500000 63.836213740688024
93600000 63.53543253824882
93700000 28.784312825688012
93800000 66.7851609618859
93900000 62.731956906798885
94000000 67.87727941057102
94100000 66.55611513380158
94200000 65.47045123060518
94300000 67.54399767436746
94400000 73.777518194779
94500000 69.50158625445768
94600000 74.29166598977032
94700000 68.01191780726381
94800000 69.32344799911

120700000 34.157682954624796
120800000 41.091094410442984
120900000 59.59140844906408
121000000 63.808863400976314
121100000 64.09087673450865
121200000 61.775262888394145
121300000 59.788421084713015
121400000 63.19222587354509
121500000 64.88992175438592
121600000 67.57508124289902
121700000 71.28513525297151
121800000 64.17497453349445
121900000 60.4263949307816
122000000 60.09588892160193
122100000 57.15770187793166
122200000 58.53458209862362
122300000 59.000453161732274
122400000 57.646015521516325
122500000 58.36394928869921
122600000 44.44092308571302
122700000 20.705300105722117
122800000 25.46119426169704
122900000 52.23567856571858
123000000 56.754767652822764
123100000 60.05683472412154
123200000 58.571775355945945
123300000 58.0195188798317
123400000 61.30017582947128
123500000 59.425908706411434
123600000 62.223496793010206
123700000 67.39310572766907
123800000 67.22758564868141
123900000 66.77900912290421
124000000 60.50790616168998
124100000 60.243067857918554
124200000

...

best so far: 0
type: [1, 2, 16, 2, 2] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 4, 1] 128
cases of this type: 32768
800000000 0.23873305790692037
best so far: 0
type: [1, 2, 32, 1, 2] 128
cases of this type: 32768
best so far: 0
type: [1, 2, 32, 2, 1] 128
cases of this type: 16384
best so far: 0
type: [1, 2, 64, 1, 1] 128
cases of this type: 8192
best so far: 0
type: [1, 4, 1, 1, 32] 128
cases of this type: 4194304
800100000 0.4978831200076149
800200000 1.3178267621125201
800300000 1.076098373807621
800400000 1.3353552742466257
800500000 0.9833069124300957
800600000 1.1578176152575599
800700000 0.9911817416267097
800800000 0.8706010757262844
800900000 0.991824082713308
801000000 0.9775530264623562
801100000 0.8149386258919756
801200000 0.961121455273135
801300000 1.0543749652252303
801400000 1.0126903190083443
801500000 0.8126884701863739
801600000 0.8996903968007691
801700000 1.080029429248454
801800000 1.0166110131329165
801900000 0.978165686402958
802000000 0.342

813800000 1.031242515874564
813900000 0.8248354573603061
814000000 0.8043839332176804
814100000 0.8941149596068567
814200000 0.9143801773731688
814300000 0.8580904762657862
814400000 0.8059802336121016
814500000 0.7926427168611666
814600000 0.8761306358767709
814700000 1.3568173897864673
814800000 0.8158033281958044
814900000 0.901321835501635
815000000 0.7951707778363168
815100000 0.8129786302027537
815200000 0.8786353893310984
815300000 0.7843577677079452
815400000 0.8927278612526265
815500000 0.809591418722403
815600000 0.9749377303941589
815700000 0.8280649932542438
815800000 0.7060286980041444
815900000 0.8588502701166087
816000000 0.8232147721080667
816100000 0.752551117932723
816200000 0.8925908870483247
816300000 0.8347733074830512
816400000 0.6962938982506872
816500000 0.9387939046085564
816600000 0.680625162273446
816700000 0.8176710527253449
816800000 0.86477370378683
816900000 0.7348959998973078
817000000 0.9215201261095345
817100000 0.7076964498321686
817200000 0.766514973

841500000 0.18477726858821975
841600000 0.24731913765848676
841700000 0.2512919663753556
841800000 0.167137249471328
best so far: 0
type: [2, 1, 1, 8, 8] 128
cases of this type: 2097152
841900000 0.2815690508094728
842000000 0.2155686652228468
842100000 0.17562009273650644
842200000 0.2593196552162707
842300000 0.17690755076584883
842400000 0.20512553207880654
842500000 0.2301937861763239
842600000 0.1703013143023703
842700000 0.3946039888943182
842800000 0.18768510725007653
842900000 0.17835213228790894
843000000 0.22566586444441214
843100000 0.1622840822915733
843200000 0.2065925996624483
843300000 0.19986730562551086
843400000 0.15442338170600137
843500000 0.23163807643842832
843600000 0.17738341029329166
843700000 0.1832332316788852
843800000 0.21649055241955983
843900000 0.1510045398990108
best so far: 0
type: [2, 1, 1, 16, 4] 128
cases of this type: 1048576
844000000 0.21444574909042516
844100000 0.19638983691460163
844200000 0.19055050233732065
844300000 0.20858257143031558
8444

861400000 0.037655596261666216
861500000 0.03134507032571435
861600000 0.01700371575497323
best so far: 0
type: [2, 2, 2, 4, 4] 128
cases of this type: 131072
861700000 0.02812937268374562
best so far: 0
type: [2, 2, 2, 8, 2] 128
cases of this type: 65536
861800000 0.021312368230375318
best so far: 0
type: [2, 2, 2, 16, 1] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 1, 8] 128
cases of this type: 131072
861900000 0.02166265686702397
best so far: 0
type: [2, 2, 4, 2, 4] 128
cases of this type: 65536
862000000 0.019691674502925076
best so far: 0
type: [2, 2, 4, 4, 2] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 8, 1] 128
cases of this type: 16384
862100000 0.0159659905649311
best so far: 0
type: [2, 2, 8, 1, 4] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 8, 2, 2] 128
cases of this type: 16384
best so far: 0
type: [2, 2, 8, 4, 1] 128
cases of this type: 8192
best so far: 0
type: [2, 2, 16, 1, 2] 128
cases of this type: 8192
best so far: 0
type

866100000 0.0006833520601027542
best so far: 0
type: [8, 1, 1, 2, 8] 128
cases of this type: 32768
best so far: 0
type: [8, 1, 1, 4, 4] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 1, 8, 2] 128
cases of this type: 8192
best so far: 0
type: [8, 1, 1, 16, 1] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 1, 8] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 2, 2, 4] 128
cases of this type: 8192
866200000 0.0002420743254005909
best so far: 0
type: [8, 1, 2, 4, 2] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 8, 1] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 1, 4] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 4, 2, 2] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 4, 1] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 1, 2] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 2, 1] 128
cases of this type: 512
best so far: 0
type: [8, 1, 16, 1, 1] 128
cases of this type: 256
best so f